In [ ]:
import pandas as pd
import numpy as np
import graphlab

In [ ]:
df_actor = pd.read_csv('../data/movie_actor_long_example')
df_actor.shape

In [ ]:
df_movie = pd.read_csv('Data/movies.csv')
with open('Data/keywords_clean.list', 'r') as f:
    data = f.read().split('\n')
data_np = np.array(data)
movie_title = df_movie.title

In [ ]:
movie_keyword = [s for s in data_np if any(xs in s for xs in movie_title[0:20])]

In [ ]:
import csv
with open('data/new_movie_keywords_example', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(movie_keyword)

In [ ]:
df_key = pd.DataFrame(movie_keyword)

In [ ]:
#add columns to name, year and key word
df_key['name'] = [x.split('\t')[0] for x in df_key[0]]
df_key['keyword'] = [x.split('\t')[-1] for x in df_key[0]]
df_key['year'] = [x.split('(')[1][:-1] for x in df_key['name']]

In [ ]:
# movie_actors = [s for s in actors_100 if any(xs in s for xs in movie_title[0:20])]

In [ ]:
from load_imdb_data import load_imdb_data

In [ ]:
cd Code

In [ ]:
filename = '../data/imdb_edges.tsv'
actors, movies = load_imdb_data(filename)

In [ ]:
movie_actors = [xs,s.values for s in movies if any(s in xs for xs in df_key['name'])]

In [ ]:
# df_key['actors'] = [s.value for s in movies if any(s in x for x in )]
movie_actors = [v for k,v in movies.iteritems() if any(k in xs for xs in movie_title[0:20])]

In [ ]:
%%timeit
m_col = []
a_col = []
for movie_t in movie_title[0:1000]:
    for k,v in movies.iteritems():
        if k in movie_t:
            for names in v:
                m_col.append(movie_t)
                a_col.append(names)
movie_actor_long = pd.concat([pd.Series(m_col), pd.Series(a_col)], axis=1)
movie_actor_long.columns = ['title','actor']
movie_actor_long.to_csv('../Data/movie_actor_long_example',index = False)

In [ ]:
%%timeit
#Make the file into pandas dataframe with movie and actor in long form
m_col = []
a_col = []
for movie_t in movie_title:
    for k,v in movies.iteritems():
        if k in movie_t:
            for names in v:
                m_col.append(movie_t)
                a_col.append(names)
movie_actor_long = pd.concat([pd.Series(m_col), pd.Series(a_col)], axis=1)
movie_actor_long.columns = ['title','actor']
movie_actor_long.to_csv('../Data/movie_actor_long_example',index = False)

In [ ]:
i = 0
for x in xrange(len(movie_title)/100):
    first = x * 100
    last = (1+x) * 100
    movie_keyword = [s for s in data_np if any(xs in s for xs in movie_title[first:last])]
    with open('../Data/new_movie_keywords_example%s' %(str(x+10)), 'wb') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(movie_keyword)
    print x*100, 'out of ',len(movie_title), x*100/float(len(movie_title))


In [ ]:
df_actor['count'] = 1
df_actor_matrix = df_actor.reset_index().pivot_table(values='count', index='title', columns='actor', aggfunc='mean')

In [ ]:
df_actor_matrix

In [ ]:
df_key = pd.read_csv('df_key_long')
df_key_matrix = df_key.reset_index().pivot_table(values='count', index='title', columns='keyword', aggfunc='mean')

In [ ]:
df_key_matrix.to_csv('df_key_matrix')

In [ ]:
df_actor_matrix.to_csv('df_actor_matrix')

In [ ]:
df_keyword = df_key[['title','keyword','count']]
keyword_sf = graphlab.SFrame(df_keyword)
m_als = graphlab.recommender.factorization_recommender.create(keyword_sf, num_factors=20,user_id = 'keyword', item_id = 'title', target = 'count', solver='als')
m_als.coefficients

In [ ]:
m_als.coefficients

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd_key = TruncatedSVD(n_components=100, random_state=42)
svd_key.fit(df_key_matrix)


In [ ]:
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
rf = RandomForestClassifier()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans

In [ ]:
# df_key_mat0 =df_key_matrix.fillna(0)
knn = KNeighborsRegressor(n_neighbors=2)
kmeans = KMeans(n_clusters=2)
# knn.fit(df_key_mat0.as_matrix())

In [ ]:
kmeans.fit_predict(df_key_mat0.as_matrix())
array_features = np.array(df_key_mat0.columns.values())
for i, cluster in enumerate(kmeans.cluster_centers_):
    idx = cluster.argsort()[0:10]
    print i, array_features[idx]

In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
import csv
# import graphlab
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans

df_key = pd.read_csv('../data/df_key_long')
df_actor = pd.read_csv('../data/movie_actor_long_example')
df_actor['count'] = 1
df_key_matrix = pd.read_csv('../data/df_key_matrix')
df_actor_matrix = pd.read_csv('../data/df_actor_matrix')

#Get the df keyword only
df_keyword = df_key[['title','keyword','count']]
# keyword_sf = graphlab.SFrame(df_keyword)
# m_als = graphlab.recommender.factorization_recommender.create(keyword_sf, user_id = 'keyword', item_id = 'title', target = 'count', solver='als')

#Use Knn to get cluters for 2 groups. And try using random forest to get highest feature importance
# knn = KNeighborsRegressor(n_neighbors=2)
# kmeans = KMeans(n_clusters=2)
# df_key_mat0 =df_key_matrix.fillna(0)
# kmeans.fit_predict(df_key_mat0.as_matrix())
# array_features = np.array(df_key_mat0.columns.values())
# for i, cluster in enumerate(kmeans.cluster_centers_):
#     idx = cluster.argsort()[0:10]
#     print i, array_features[idx]

##That is the start from Tuesday!!!

In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
import csv

In [4]:
df_key = pd.read_csv('../data/df_key_long')

/Users/zoesh/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_key_matrix = pd.read_csv('../data/df_key_matrix')

In [5]:
df_key_matrix = df_key.reset_index().pivot_table(values='count', index='title', columns='keyword', aggfunc='mean')


In [7]:

df_key_mat_ex = df_key_matrix[df_key_matrix.columns[df_key_matrix.sum()>5]]

In [24]:
df_key_mat_ex.fillna(0,inplace = True)

/Users/zoesh/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2705: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [9]:
df_key_mat_ex.shape

(20939, 19718)

In [10]:
# df_actor = pd.read_csv('../data/movie_actor_long_example')
# df_actor['count'] = 1
# df_actor_matrix = df_actor.reset_index().pivot_table(values='count', index='title', columns='keyword', aggfunc='mean')
df_key_matrix['year'] = [x.split('(')[1][:-1] for x in df_key_matrix.index.values]
df_year = pd.get_dummies(df_key_matrix['year'])

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris

In [25]:
clf = DecisionTreeClassifier(max_depth = 20)
clf.fit(df_key_mat_ex.values,np.array(df_key_mat_ex.index))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [29]:
from IPython.display import Image  
from sklearn.externals.six import StringIO
from sklearn import tree
with open("../data/movie.dot", 'w') as f:
    f = tree.export_graphviz(clf, out_file=f)
# dot_data = StringIO()  
# DecisionTreeClassifier.tree.export_graphviz(clf, out_file=dot_data,  
#                          feature_names=iris.feature_names,  
#                          class_names=iris.target_names,  
#                          filled=True, rounded=True,  
#                          special_characters=True)  
# >>> graph = pydot.graph_from_dot_data(dot_data.getvalue())  
# >>> Image(graph.create_png())  

In [28]:
import os
os.unlink("../data/movie.dot")


In [30]:
clf.score(df_key_mat_ex.values,np.array(df_key_mat_ex.index))

0.012130474234681695

In [32]:
clf40 = DecisionTreeClassifier(max_depth = 140)
clf40.fit(df_key_mat_ex.values,np.array(df_key_mat_ex.index))
clf40.score(df_key_mat_ex.values,np.array(df_key_mat_ex.index))

0.1621853956731458

In [57]:
df = df_key_mat_ex[df_key_mat_ex.sum(axis = 1) > 5]

In [60]:
clf40 = DecisionTreeClassifier(max_depth = 20)
clf40.fit(df.values,np.array(df.index))
clf40.score(df.values,np.array(df.index))

0.016508254127063533

In [67]:
yr_cnt = Counter(df_key_matrix['year']).most_common()

In [93]:
year_list = [i[0] for i in yr_cnt if i[1]>5]

In [2]:
import numpy as np
import pandas as pd
from scipy import sparse
import csv
# import graphlab
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans
import os
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Image  
from sklearn.externals.six import StringIO
from sklearn import tree
from collections import Counter

In [ ]:
df_key = pd.read_csv('../data/df_key_long',dtype={'title': 'str','keyword': 'S20','count':'int'})


In [21]:
df_key_matrix = df_key.reset_index().pivot_table(values='count', index='title', columns='keyword', aggfunc='mean')
df_key_matrix['year'] = [x.split('(')[1][:-1] for x in df_key_matrix.index.values]
#Better to use this one!
yr_cnt = Counter(df_key_matrix['year']).most_common()
year_list = [i[0] for i in yr_cnt if i[1]>5]
df_key_matrix = df_key_matrix[df_key_matrix['year'].isin(year_list)]
df_key_matrix['year'] = df_key_matrix['year'].astype(int)
df_key_mat_ex = df_key_matrix[df_key_matrix.columns[df_key_matrix.sum()>5]].fillna(0)

# list_of_values = [3,6]

# y = df[df['A'] in list_of_values]

ValueError: Item wrong length 81206 instead of 20874.

In [53]:
test = df.pop('year')

In [59]:
df = df[df.sum(axis = 1) > 5]

In [65]:
df = df.join(test)

In [99]:
# df['after2000'] = (df.year>2000).astype(int)
clf = DecisionTreeClassifier('entropy',max_depth = 20)
clf.fit(df.values,np.array(df.index))
clf.score(df.values,np.array(df.index))

0.87592430129088861

In [101]:
with open("../data/movie4.dot", 'w') as f:
    f = tree.export_graphviz(clf, out_file=f)

In [105]:
from sklearn.externals import joblib
joblib.dump(clf, 'my_movie_model.pkl')
# >>> from sklearn.externals import joblib
# >>> model_clone = joblib.load('my_movie.model.pkl')

['my_movie_model.pkl',
 'my_movie_model.pkl_01.npy',
 'my_movie_model.pkl_02.npy',
 'my_movie_model.pkl_03.npy',
 'my_movie_model.pkl_04.npy']